In [1]:
import random
import gurobipy as gp
from gurobipy import GRB
import copy
import numpy as np

n=5
def generate_data(n):
    # Initializing an empty adjacency matrix with zeros
    W = [[0] * n for _ in range(n)]

    # Generate random edge weights (assuming non-negative weights)
    for i in range(n):
        for j in range(i+1, n):
            t = random.randint(0,2)
            if t == 0:
                weight = 0
            else:
                weight = random.randint(1,10)
            # weight = random.uniform(0, 2)  # Adjust the range as needed
            # Ensure that edges are undirected by setting W[i][j] = W[j][i] = weight
            W[i][j] = W[j][i] = weight
    return W

In [2]:
def BP(W,n):
    # Create a new model
    m = gp.Model("mip1")
    # Create variable

    x = {}
    x = m.addVars(range(n),vtype = "B", name="x")
    y = m.addVars(range(n),range(n),vtype = "B", name="y")

    m.setObjective(sum(W[i][j]*y[i,j] for i in range(n) for j in range(n)), GRB.MAXIMIZE)
    m.addConstrs(y[i,j]<=(x[i]+x[j]) for i in range(n) for j in range(n))
    m.addConstrs(y[i,j]+x[i]+x[j]<=2 for i in range(n) for j in range(n))
    m.optimize()
    y_result=[]

    for v in x.values():
        if v.X == 0 or v.X==-0:
            y_result.append(0)
        else:
            y_result.append(1)
    return y_result

In [3]:
def count_weight(W:list, A:list, B:list):
    total_weight = 0
    for i in A:
        for j in B:
            total_weight += W[i][j]
    return total_weight

In [4]:
def weight_change(W, A, B, current_weight):
    weight_change_list = []
    for i in range(n):
        temp_A = A.copy()
        temp_B = B.copy()
        if i in A:

            temp_B.append(i)
            temp_A.remove(i)
        else:

            temp_A.append(i)
            temp_B.remove(i)
        new_weight = count_weight(W, temp_A, temp_B)
        result = new_weight - current_weight
        weight_change_list.append(result)
    return weight_change_list

In [5]:
def random_set():
    A = []
    B = []
    xdata_gh = []

    for i in range(n):
        r = random.randint(0,1)
        if r == 0:
            xdata_gh.append(0)
            A.append(i)
        else:
            xdata_gh.append(1)
            B.append(i)
    return xdata_gh, A, B

In [6]:
def GH(W, A, B, n):
    total_weight = count_weight(W, A, B)

    for i in range(n):
        temp_A = A.copy()
        temp_B = B.copy()
        if i in A:
            temp_B.append(i)
            temp_A.remove(i)
        else:
            temp_A.append(i)
            temp_B.remove(i)
        new_weight = count_weight(W, temp_A, temp_B)
        if new_weight >= total_weight:
            A = temp_A
            B = temp_B
            total_weight = new_weight
    A = sorted(A)
    B = sorted(B)
    total_weight = count_weight(W, A, B)
    return total_weight, A, B

In [7]:
def get_data(k):
    A_GH_list = []
    B_GH_list = []
    gh_weight_list = []
    raw_data = []
    xdata1_l = []
    ydata1_l = []

    for i in range(k):
        sum_W = []
        data_list = []
        data = generate_data(n)

        xdata_gh, A, B = random_set()
        gh_weight, A_GH, B_GH = GH(data,A,B,n)

        A_GH_list.append(A_GH)
        B_GH_list.append(B_GH)
        gh_weight_list.append(gh_weight)

        current_weight = count_weight(data,A,B)
        weight_diff = weight_change(data,A,B,current_weight)

        result = BP(data, n)
        ydata1_l.append(result)
        
        for j in data:
            sum_W.append(sum(j))
        data_list.append(sum_W)
        data_list.append(xdata_gh)
        # data_list.append(weight_diff)
        xdata1_l.append(data_list)
        raw_data.append(data)

    return xdata1_l, ydata1_l, A_GH_list, B_GH_list, gh_weight_list, raw_data

In [8]:
k1 = 500
xtrain_list, ytrain_list, A_GH_train, B_GH_train, gh_weight_train, raw_data = get_data(k1)

k2 = 100
xtest_list, ytest_list, A_GH_test, B_GH_test, gh_weight_test, raw_data = get_data(k2)

Restricted license - for non-production use only - expires 2024-10-28
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 50 rows, 30 columns and 140 nonzeros
Model fingerprint: 0x2790498e
Variable types: 0 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 9e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 36 rows and 18 columns
Presolve time: 0.00s
Presolved: 14 rows, 12 columns, 42 nonzeros
Variable types: 0 continuous, 12 integer (12 binary)
Found heuristic solution: objective 64.0000000

Root relaxation: objective 8.600000e+01, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     

In [9]:
A_BP = []
B_BP = []

for i in ytest_list:
    temp_A = []
    temp_B = []
    for j in range(len(i)):
        if i[j]>0.5:
            temp_A.append(j)
        else:
            temp_B.append(j)
    A_BP.append(temp_A)
    B_BP.append(temp_B)

In [10]:
BP_result = 0
GH_result = 0
for i in range(len(A_BP)):
    GH_weight = count_weight(raw_data[i], A_GH_test[i], B_GH_test[i])
    BP_weight = count_weight(raw_data[i], A_BP[i], B_BP[i])
    if BP_weight >= GH_weight:
        BP_result += 1
    else:
        GH_result += 1

print(f"Outcome of the BP where they have a higher performance is {BP_result/len(A_BP)}%")
print(f"Outcome of the GH where they have a higher performance is {GH_result/len(A_BP)}%")

Outcome of the BP where they have a higher performance is 1.0%
Outcome of the GH where they have a higher performance is 0.0%


In [11]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score

def build_binary_model(xtrain, ytrain, xtest, ytest, nrlayers, nrnodes, rate, k):
    # This function builds the model with nrlayes layers and nrnodes nodes in each layer
    # nrlayers is a number and nrnodes is a vector specifying how many nodes each layer should contain
    model = keras.Sequential()
    numclasses = k

    # add the layers
    for i in range(nrlayers + 1):
        if (i == 0):
            # input layer
            model.add(keras.Input(shape=input_shape))
        else:
            # hidden layers
            nrnode = nrnodes[i - 1]
            model.add(keras.layers.Dense(nrnode, activation="sigmoid"))

    model.add(keras.layers.Dense(numclasses, activation="sigmoid"))

    # give summary of model
    model.summary()

    # train model
    batch_size = 30 # Based on SGD --> 1 gradient is calculated using 32 data points every time
    #so in each epoch, training_size/batch_size many gradient calculations and weight updates
    epochs = 10 # How many times to go through the data to complete the SGD

    opt = keras.optimizers.SGD(learning_rate=rate)
    model.compile(loss="binary_crossentropy",
                  optimizer=opt,
                  metrics=["accuracy"])
    
    model.fit(xtrain, ytrain, batch_size=batch_size, epochs=epochs, validation_split=0.1,)
    score_train = model.evaluate(xtrain, ytrain, verbose=1)
    print(f"Train loss:, {score_train[0]}, Train accuracy:, {score_train[1]}")
    # get results
    score = model.evaluate(xtest, ytest, verbose=1)
    print(f"Test loss: {score[0]}, Test accuracy:, {score[1]}")
    return model

In [12]:
xtrain = np.array(xtrain_list)
xtest = np.array(xtest_list)
ytrain = np.array(ytrain_list)
ytest = np.array(ytest_list)

In [13]:
xtrain=np.reshape(xtrain,[k1, 2*n])
xtest=np.reshape(xtest,[k2, 2*n])

input_shape = (2*n, )
models1 = build_binary_model(xtrain, ytrain, xtest, ytest, 4, [2*n,1.75*n, 1.5*n, 1.25*n], 0.02, n)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                110       
                                                                 
 dense_1 (Dense)             (None, 8)                 88        
                                                                 
 dense_2 (Dense)             (None, 7)                 63        
                                                                 
 dense_3 (Dense)             (None, 6)                 48        
                                                                 
 dense_4 (Dense)             (None, 5)                 35        
                                                                 
Total params: 344 (1.34 KB)
Trainable params: 344 (1.34 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
15/15 [========

In [14]:
models1.predict(xtrain)

16/16 [==============================] - 0s 2ms/step


array([[0.50346047, 0.43587   , 0.5203141 , 0.44743362, 0.5193768 ],
       [0.5042036 , 0.4368468 , 0.5199221 , 0.44654042, 0.51898193],
       [0.5025471 , 0.43632716, 0.5199798 , 0.44773602, 0.5187669 ],
       ...,
       [0.50312096, 0.4361662 , 0.5202498 , 0.4477226 , 0.5191499 ],
       [0.50402284, 0.43618542, 0.52023894, 0.44703558, 0.5193325 ],
       [0.50426096, 0.4368139 , 0.51994056, 0.44651845, 0.5190153 ]],
      dtype=float32)

In [15]:
NN = models1.predict(xtest)
NN_ytest = NN.tolist()
NN_ytest = [[int(x) for x in sublist] for sublist in NN_ytest]

4/4 [==============================] - 0s 3ms/step


In [16]:
A_NN = []
B_NN = []

for i in NN_ytest:
    temp_A = []
    temp_B = []
    for j in range(len(i)):
        if i[j]>0.5:
            temp_A.append(j)
        else:
            temp_B.append(j)
    A_NN.append(temp_A)
    B_NN.append(temp_B)

In [17]:
NN_result = 0
GH_result = 0
for i in range(len(A_NN)):
    GH_weight = count_weight(raw_data[i], A_GH_test[i], B_GH_test[i])
    NN_weight = count_weight(raw_data[i], A_NN[i], B_NN[i])

    if NN_weight >= GH_weight:
        NN_result += 1
    else:
        GH_result += 1

print(f"Outcome of the MLGH where they have a higher performance is {NN_result/len(A_NN)}%")
print(f"Outcome of the GH where they have a higher performance is {GH_result/len(A_NN)}%")

Outcome of the MLGH where they have a higher performance is 0.0%
Outcome of the GH where they have a higher performance is 1.0%
